In [1]:
import pandas as pd
import csv
import numpy as np

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
                
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            

def filterPair(p):
    return len(p[0]) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def getChars(item):
    return [element for element in item]

In [3]:
data_orig = pd.read_csv('../input/ru_with_types/output-00001-of-00100', sep='\t', names=['class', 'before', 'after'],
                       quoting=csv.QUOTE_NONE, encoding='utf-8', dtype=str)

In [4]:
data_train = pd.read_csv('../input/ru_train.csv', dtype = str)
data_test = pd.read_csv('../input/ru_test.csv', dtype = str)

In [5]:
MAX_LENGTH = 100

In [6]:
big_str = list(data_orig.before.astype(str).values)
output_list = list(data_orig.after.astype(str).values)

stride = 3
input_list = []
pairs = []
for i in range(len(big_str)):
    cur_item = ['<norm>'] + getChars(big_str[i]) + ['</norm>']
    cur_item = getChars(' '.join(big_str[i-stride:i])) \
    + cur_item + \
    getChars(' '.join(big_str[i+1:i+stride+1]))
    input_list += [' '.join(cur_item)]

pairs = list(zip(input_list, output_list))
print('Len pairs before filtering:', len(pairs))
pairs = filterPairs(pairs)
print('Len pairs after filtering:', len(pairs))

Len pairs before filtering: 2981069
Len pairs after filtering: 1887522


In [8]:
input_lang, output_lang = Lang('nonnorm'), Lang('norm')

for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        for i in range(self.n_layers):
            output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result
        
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=2):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result
        
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_output, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)))
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]))
        return output, hidden, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [12]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes).view(-1, 1))
    if use_cuda:
        return result.cuda()
    else:
        return result


def variablesFromPair(pair):
    input_variable = variableFromSentence(input_lang, pair[0])
    target_variable = variableFromSentence(output_lang, pair[1])
    return (input_variable, target_variable)

In [13]:
teacher_forcing_ratio = 0.5


def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]
    
    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
   
    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_output, encoder_outputs)
            loss += criterion(decoder_output, target_variable[di])
            decoder_input = target_variable[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_output, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]
            
            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            
            loss += criterion(decoder_output, target_variable[di])
            if ni == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0] / target_length

In [14]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [15]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [variablesFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_variable = training_pair[0]
        target_variable = training_pair[1]
 
        loss = train(input_variable, target_variable, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words,
                               1, dropout_p=0.1)

if use_cuda:
    print('Using CUDA')
    encoder1 = encoder1.cuda()
    attn_decoder1 = attn_decoder1.cuda()

trainIters(encoder1, attn_decoder1, 75000, print_every=100)

Using CUDA
0m 10s (- 133m 37s) (100 0%) 2.4801
0m 15s (- 98m 49s) (200 0%) 1.3551
0m 20s (- 86m 41s) (300 0%) 0.6106
0m 26s (- 81m 0s) (400 0%) 1.1455
0m 31s (- 76m 59s) (500 0%) 1.1940
0m 35s (- 74m 1s) (600 0%) 1.2815
0m 40s (- 72m 17s) (700 0%) 1.0909
0m 46s (- 71m 14s) (800 1%) 1.0254
0m 50s (- 69m 55s) (900 1%) 1.2471
0m 55s (- 69m 2s) (1000 1%) 1.2285
1m 0s (- 68m 14s) (1100 1%) 1.1218
1m 6s (- 67m 53s) (1200 1%) 1.1244
1m 11s (- 67m 8s) (1300 1%) 1.1291
1m 16s (- 66m 51s) (1400 1%) 1.0780
1m 21s (- 66m 28s) (1500 2%) 1.1007
1m 26s (- 66m 14s) (1600 2%) 0.9705
1m 31s (- 65m 53s) (1700 2%) 1.1884
1m 36s (- 65m 29s) (1800 2%) 1.0177
1m 42s (- 65m 25s) (1900 2%) 1.1489
1m 46s (- 65m 4s) (2000 2%) 1.2692
1m 52s (- 64m 48s) (2100 2%) 1.2382
1m 56s (- 64m 28s) (2200 2%) 1.2546
2m 2s (- 64m 22s) (2300 3%) 1.0099
2m 7s (- 64m 7s) (2400 3%) 1.2606
2m 12s (- 63m 50s) (2500 3%) 1.3285
2m 17s (- 63m 37s) (2600 3%) 1.1326
2m 22s (- 63m 32s) (2700 3%) 1.2358
2m 27s (- 63m 18s) (2800 3%) 0.9611

18m 47s (- 44m 59s) (22100 29%) 0.4353
18m 53s (- 44m 54s) (22200 29%) 0.3153
18m 58s (- 44m 50s) (22300 29%) 0.3692
19m 4s (- 44m 46s) (22400 29%) 0.3707
19m 9s (- 44m 41s) (22500 30%) 0.3020
19m 14s (- 44m 37s) (22600 30%) 0.4740
19m 19s (- 44m 31s) (22700 30%) 0.3846
19m 24s (- 44m 26s) (22800 30%) 0.6881
19m 29s (- 44m 21s) (22900 30%) 0.4772
19m 35s (- 44m 16s) (23000 30%) 0.4574
19m 39s (- 44m 11s) (23100 30%) 0.5055
19m 44s (- 44m 5s) (23200 30%) 0.4475
19m 49s (- 44m 0s) (23300 31%) 0.4374
19m 55s (- 43m 55s) (23400 31%) 0.4420
20m 0s (- 43m 50s) (23500 31%) 0.5252
20m 5s (- 43m 44s) (23600 31%) 0.4532
20m 10s (- 43m 39s) (23700 31%) 0.3466
20m 15s (- 43m 35s) (23800 31%) 0.6021
20m 20s (- 43m 30s) (23900 31%) 0.5455
20m 26s (- 43m 25s) (24000 32%) 0.4595
20m 31s (- 43m 20s) (24100 32%) 0.3337
20m 36s (- 43m 15s) (24200 32%) 0.4111
20m 41s (- 43m 10s) (24300 32%) 0.4415
20m 46s (- 43m 4s) (24400 32%) 0.3847
20m 51s (- 43m 0s) (24500 32%) 0.4132
20m 56s (- 42m 55s) (24600 32%) 0

In [210]:
ind_to_test = 5

print(input_list[ind_to_test])
evaluate(encoder1, attn_decoder1, input_list[ind_to_test])[0]

в Валуйском районе <norm> , </norm> Белгородская область ,


RuntimeError: invalid argument 6: wrong matrix size at /pytorch/torch/lib/THC/generic/THCTensorMathBlas.cu:453

In [212]:
test_pairs = pairs[-1000:]

preds = [evaluate(encoder1, attn_decoder1, item[0])[0] for item in test_pairs]

KeyboardInterrupt: 

In [ ]:
preds

In [211]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    input_variable = variableFromSentence(input_lang, sentence)
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                 encoder_hidden)
        encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))  # SOS
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_output, encoder_outputs)
        decoder_attentions[di] = decoder_attention.data
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni])
        
        decoder_input = Variable(torch.LongTensor([[ni]]))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    return decoded_words, decoder_attentions[:di + 1]

In [208]:
ind_to_test = 5

print(input_list[ind_to_test])
evaluate(encoder1, attn_decoder1, input_list[ind_to_test])[0]

в Валуйском районе <norm> , </norm> Белгородская область ,


RuntimeError: invalid argument 6: wrong matrix size at /pytorch/torch/lib/THC/generic/THCTensorMathBlas.cu:453

In [174]:
input_list[0]

' <norm> Ныне </norm> — в Валуйском'